# Section 3.2: Best Practices for Quantum Circuit Design

## Engineering production-ready quantum circuits with hardware awareness and optimization

In this section, you'll learn the engineering discipline required to design quantum circuits that work on real hardware. We'll cover hardware constraints, parameterization strategies, modular design patterns, and optimization techniques that separate prototype code from production-ready implementations.

## Learning Objectives

By the end of this section, you will be able to:

- Validate circuits against real hardware connectivity constraints
- Design circuits using native gate sets for compilation efficiency
- Apply parameterization strategies for variational algorithms
- Build modular, reusable circuit components
- Optimize circuit depth and gate count
- Account for decoherence times (T1/T2) in circuit design
- Follow best practices for production-ready quantum code

In [ ]:
import cirq
import numpy as np
import sympy
import matplotlib.pyplot as plt
from typing import Dict, List

## 1. Hardware Connectivity Constraints

Real quantum hardware has limited qubit connectivity. Not every qubit can interact directly with every other qubit. Violating connectivity constraints means your circuit won't compile to hardware.

In [ ]:
print("Hardware Connectivity Validation")
print("Real quantum processors have limited qubit connectivity\n")

# Define linear connectivity (common architecture)
connectivity = {
    0: [1],        # q0 connects to q1
    1: [0, 2],     # q1 connects to q0 and q2
    2: [1]         # q2 connects to q1
}

print("Connectivity topology: 0 -- 1 -- 2 (linear chain)")

q0, q1, q2 = cirq.LineQubit.range(3)

# Valid circuit respects connectivity
valid_circuit = cirq.Circuit(
    cirq.CNOT(q0, q1),  # Adjacent qubits
    cirq.CNOT(q1, q2)   # Adjacent qubits
)

print("\nValid circuit (respects connectivity):")
print(valid_circuit)

# Invalid circuit violates connectivity
invalid_circuit = cirq.Circuit(
    cirq.CNOT(q0, q2)   # NOT adjacent - violates connectivity!
)

print("\nInvalid circuit (violates connectivity):")
print(invalid_circuit)

# Validation function
def validate_connectivity(circuit: cirq.Circuit, connectivity: Dict[int, List[int]]) -> bool:
    """Check if circuit respects qubit connectivity."""
    for moment in circuit:
        for op in moment:
            if len(op.qubits) == 2:
                q0_idx = op.qubits[0].x
                q1_idx = op.qubits[1].x
                if q1_idx not in connectivity.get(q0_idx, []):
                    return False
    return True

print(f"\nValid circuit passes validation: {validate_connectivity(valid_circuit, connectivity)}")
print(f"Invalid circuit passes validation: {validate_connectivity(invalid_circuit, connectivity)}")
print("\nNote: Invalid circuits require SWAP gates for routing, increasing depth!")

## 2. Circuit Depth and Decoherence Time Constraints

Circuit depth (number of time steps) must respect decoherence times. Rule of thumb: execution time should be less than T2/10 to maintain quantum coherence.

In [ ]:
print("Circuit Depth and Decoherence Constraints")
print("Execution time must be much shorter than T1/T2 times\n")

q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.H(q1),
    cirq.CNOT(q1, q2),
    cirq.measure(q0, q1, q2, key='result')
)

print("Example circuit:")
print(circuit)

# Calculate circuit depth
depth = len(circuit) - 1  # Exclude measurement
print(f"\nCircuit depth: {depth} moments")

# Typical gate times for superconducting qubits
gate_times = {
    'single': 25e-9,  # 25 nanoseconds
    'two': 50e-9      # 50 nanoseconds
}

# Estimate execution time
total_time = 0.0
for moment in circuit[:-1]:  # Exclude measurement
    for op in moment:
        if len(op.qubits) == 1:
            total_time += gate_times['single']
        elif len(op.qubits) == 2:
            total_time += gate_times['two']

print(f"Estimated execution time: {total_time * 1e9:.1f} ns")

# Typical decoherence times
t1 = 50e-6  # 50 microseconds (amplitude damping)
t2 = 30e-6  # 30 microseconds (phase damping)

print(f"\nTypical superconducting qubit times:")
print(f"  T1 (amplitude damping): {t1 * 1e6:.0f} μs")
print(f"  T2 (phase damping): {t2 * 1e6:.0f} μs")

# Check constraint: execution_time << T2/10
constraint_time = t2 / 10.0
passes_constraint = total_time < constraint_time

print(f"\nRule of thumb: execution_time < T2/10 = {constraint_time * 1e6:.1f} μs")
print(f"Circuit passes constraint: {passes_constraint}")
print(f"Safety margin: {constraint_time / total_time:.0f}x")

## 3. Native Gate Sets and Circuit Compilation

Each quantum processor has a native gate set. Arbitrary gates must be decomposed into native gates. Understanding this avoids compilation surprises.

In [ ]:
print("Native Gate Sets and Compilation")
print("Different hardware supports different native gates\n")

print("Common native gate sets:")
print("  Google: sqrt(X), sqrt(Y), PhasedXZ, sqrt(iSWAP), CZ")
print("  IBM: RZ, sqrt(X), CNOT")
print("  IonQ: arbitrary single-qubit rotations, XX gate\n")

q0, q1 = cirq.LineQubit.range(2)

# High-level circuit using arbitrary gates
high_level = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.T(q1),
    cirq.SWAP(q0, q1)
)

print("High-level circuit (arbitrary gates):")
print(high_level)
print(f"Gate count: {len(list(high_level.all_operations()))}")
print(f"Depth: {len(high_level)}")

# Decompose to simpler gates
compiled = cirq.Circuit()
for moment in high_level:
    for op in moment:
        # Decompose each operation
        decomposed = cirq.decompose(op, keep=lambda op: isinstance(
            op.gate,
            (cirq.XPowGate, cirq.YPowGate, cirq.ZPowGate, cirq.CZPowGate)
        ))
        compiled.append(decomposed)

print("\nCompiled circuit (native gates):")
print(compiled)
print(f"Gate count: {len(list(compiled.all_operations()))}")
print(f"Depth: {len(compiled)}")

print("\nNote: SWAP gate decomposes to 3 CNOTs!")
print("This significantly impacts circuit depth and fidelity.")

## 4. Parameterization for Variational Algorithms

Use symbolic parameters (sympy.Symbol) instead of hardcoded values. This enables parameter sweeps, optimization, and flexible circuit design.

In [ ]:
print("Parameterization with Symbolic Parameters")
print("Use sympy.Symbol for flexible, optimizable circuits\n")

q = cirq.LineQubit(0)

# BAD: Hardcoded value
hardcoded = cirq.Circuit(
    cirq.ry(0.5)(q)  # Fixed angle - can't optimize!
)

print("Hardcoded circuit:")
print(hardcoded)
print(f"Is parameterized: {cirq.is_parameterized(hardcoded)}")

# GOOD: Symbolic parameter
theta = sympy.Symbol('theta')
parameterized = cirq.Circuit(
    cirq.ry(theta)(q)  # Symbolic - ready for optimization!
)

print("\nParameterized circuit:")
print(parameterized)
print(f"Is parameterized: {cirq.is_parameterized(parameterized)}")

# Parameter resolution
print("\nResolving parameter values:")
for angle in [0, np.pi/4, np.pi/2]:
    resolved = cirq.resolve_parameters(parameterized, {'theta': angle})
    print(f"  theta = {angle:.4f}: {resolved[0]}")

print("\nKey benefit: Same circuit, many parameter values!")
print("Essential for VQE, QAOA, quantum machine learning.")

## 5. Multi-Parameter Circuits

Variational algorithms often require multiple independent parameters. Each parameter can be optimized separately or jointly.

In [ ]:
print("Multi-Parameter Circuits")
print("Each parameter provides a degree of freedom for optimization\n")

q0, q1 = cirq.LineQubit.range(2)

# Define multiple independent parameters
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')

# Build multi-parameter circuit
circuit = cirq.Circuit(
    cirq.ry(alpha)(q0),
    cirq.ry(beta)(q1),
    cirq.CNOT(q0, q1),
    cirq.rz(gamma)(q1),
    cirq.CNOT(q0, q1)
)

print("Multi-parameter circuit:")
print(circuit)

# Extract parameter names
param_names = cirq.parameter_names(circuit)
print(f"\nParameters: {sorted(param_names)}")
print(f"Number of parameters: {len(param_names)}")

# Resolve with specific values
param_values = {
    'alpha': np.pi/4,
    'beta': np.pi/3,
    'gamma': np.pi/2
}

resolved = cirq.resolve_parameters(circuit, param_values)
print("\nCircuit with resolved parameters:")
print(resolved)
print(f"Is still parameterized: {cirq.is_parameterized(resolved)}")

## 6. Parameter Reuse and Symmetric Circuits

Sometimes you want the same parameter value across multiple gates. This creates symmetric structures and reduces the optimization search space.

In [ ]:
print("Parameter Reuse for Symmetric Circuits")
print("Same parameter applied to multiple gates\n")

qubits = cirq.LineQubit.range(3)
theta = sympy.Symbol('theta')

# Reuse theta across all qubits
symmetric_circuit = cirq.Circuit(
    [cirq.ry(theta)(q) for q in qubits]
)

print("Symmetric circuit (parameter reuse):")
print(symmetric_circuit)

param_names = cirq.parameter_names(symmetric_circuit)
print(f"\nNumber of unique parameters: {len(param_names)}")
print(f"Parameter name: {param_names}")
print(f"Number of gates using this parameter: 3")

print("\nBenefits of parameter reuse:")
print("  - Smaller optimization search space")
print("  - Enforces symmetry in ansatz")
print("  - Useful for identical qubit rotations")

# Resolve single parameter affects all gates
resolved = cirq.resolve_parameters(symmetric_circuit, {'theta': np.pi/2})
print("\nAfter resolving theta = π/2:")
print(resolved)
print("All three rotations use the same angle!")

## 7. Modular Circuit Design with Reusable Subcircuits

Build complex circuits from simple, reusable building blocks. This improves code maintainability and reduces bugs.

In [ ]:
print("Modular Circuit Design")
print("Compose complex circuits from reusable building blocks\n")

# Define reusable subcircuits
def bell_state_subcircuit(q0: cirq.Qid, q1: cirq.Qid) -> cirq.Circuit:
    """Prepare Bell state (|00⟩ + |11⟩)/√2."""
    return cirq.Circuit(
        cirq.H(q0),
        cirq.CNOT(q0, q1)
    )

def controlled_rotation_block(control: cirq.Qid, target: cirq.Qid, angle: float) -> cirq.Circuit:
    """Apply controlled Z-rotation."""
    return cirq.Circuit(
        cirq.CNOT(control, target),
        cirq.rz(angle)(target),
        cirq.CNOT(control, target)
    )

print("Building block 1: Bell state preparation")
q0, q1 = cirq.LineQubit.range(2)
bell = bell_state_subcircuit(q0, q1)
print(bell)

print("\nBuilding block 2: Controlled rotation")
ctrl_rot = controlled_rotation_block(q0, q1, np.pi/4)
print(ctrl_rot)

# Compose larger circuit from blocks
print("\nComposed circuit using building blocks:")
composed = cirq.Circuit()
composed += bell_state_subcircuit(q0, q1)
composed += controlled_rotation_block(q0, q1, np.pi/4)
print(composed)

print("\nAdvantages of modular design:")
print("  - Reusable components across projects")
print("  - Easier testing (test each block separately)")
print("  - Clear separation of concerns")
print("  - Maintainable and readable code")

## 8. QAOA Circuit Layers as Modular Components

QAOA (Quantum Approximate Optimization Algorithm) uses repeated layers. Each layer is a perfect example of modular design.

In [ ]:
print("QAOA Layers as Modular Building Blocks")
print("Each QAOA layer has cost + mixer components\n")

def qaoa_layer(qubits: List[cirq.Qid], gamma: sympy.Symbol, beta: sympy.Symbol) -> cirq.Circuit:
    """Create single QAOA layer with cost and mixer."""
    circuit = cirq.Circuit()
    
    # Cost layer: ZZ interactions between adjacent qubits
    for i in range(len(qubits) - 1):
        circuit.append([
            cirq.CNOT(qubits[i], qubits[i + 1]),
            cirq.rz(2 * gamma)(qubits[i + 1]),
            cirq.CNOT(qubits[i], qubits[i + 1])
        ])
    
    # Mixer layer: X rotations on all qubits
    for qubit in qubits:
        circuit.append(cirq.rx(-2 * beta)(qubit))
    
    return circuit

qubits = cirq.LineQubit.range(3)
gamma = sympy.Symbol('gamma')
beta = sympy.Symbol('beta')

# Single QAOA layer
layer = qaoa_layer(qubits, gamma, beta)
print("Single QAOA layer:")
print(layer)

# Build p-layer QAOA circuit
print("\nMulti-layer QAOA (p=2):")
p = 2
qaoa_circuit = cirq.Circuit()

# Initial state: uniform superposition
qaoa_circuit.append(cirq.H.on_each(*qubits))

# Add p layers with different parameters
for layer_idx in range(p):
    gamma_i = sympy.Symbol(f'gamma_{layer_idx}')
    beta_i = sympy.Symbol(f'beta_{layer_idx}')
    qaoa_circuit += qaoa_layer(qubits, gamma_i, beta_i)

print(qaoa_circuit)
print(f"\nTotal parameters: {len(cirq.parameter_names(qaoa_circuit))}")
print("Depth scales linearly with p (number of layers)")

## 9. Circuit Depth Optimization

Reducing circuit depth is critical for NISQ devices. Use EARLIEST insert strategy to parallelize gates that act on different qubits.

In [ ]:
print("Circuit Depth Optimization")
print("Parallelize independent gates to reduce depth\n")

q0, q1, q2 = cirq.LineQubit.range(3)

# Unoptimized: sequential construction
unoptimized = cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.X(q0),
    cirq.Y(q1),
    cirq.Z(q2)
)

print("Unoptimized circuit (default NEW strategy):")
print(unoptimized)
print(f"Depth: {len(unoptimized)} moments")

# Optimized: use EARLIEST strategy
optimized = cirq.Circuit()
for moment in unoptimized:
    for op in moment:
        optimized.append(op, strategy=cirq.InsertStrategy.EARLIEST)

print("\nOptimized circuit (EARLIEST strategy):")
print(optimized)
print(f"Depth: {len(optimized)} moments")

depth_reduction = len(unoptimized) - len(optimized)
print(f"\nDepth reduction: {depth_reduction} moments")
print(f"Improvement: {depth_reduction/len(unoptimized)*100:.1f}%")
print("\nKey insight: Gates on different qubits can run in parallel!")

## 10. Gate Cancellation and Merging

Adjacent inverse gates cancel (X·X = I, H·H = I). Adjacent single-qubit gates can be merged into a single gate. Use Cirq's optimization utilities.

In [ ]:
print("Gate Cancellation and Merging")
print("Remove redundant gates and merge single-qubit operations\n")

q0 = cirq.LineQubit(0)

# Circuit with inverse gates
circuit_with_inverses = cirq.Circuit(
    cirq.X(q0),
    cirq.X(q0),  # X·X = I (cancels)
    cirq.H(q0),
    cirq.H(q0),  # H·H = I (cancels)
    cirq.Y(q0)
)

print("Circuit with inverse gates:")
print(circuit_with_inverses)
print(f"Gate count: {len(list(circuit_with_inverses.all_operations()))}")

# Apply optimization
optimized = cirq.drop_empty_moments(
    cirq.drop_negligible_operations(circuit_with_inverses)
)

print("\nAfter gate cancellation:")
print(optimized)
print(f"Gate count: {len(list(optimized.all_operations()))}")

# Merge adjacent single-qubit gates
circuit_to_merge = cirq.Circuit(
    cirq.H(q0),
    cirq.Z(q0),
    cirq.H(q0)
)

print("\nCircuit with multiple single-qubit gates:")
print(circuit_to_merge)
print(f"Gate count: {len(list(circuit_to_merge.all_operations()))}")

merged = cirq.merge_single_qubit_gates_to_phased_x_and_z(circuit_to_merge)

print("\nAfter merging single-qubit gates:")
print(merged)
print(f"Gate count: {len(list(merged.all_operations()))}")
print("\nNote: H·Z·H = X (three gates merged to one equivalent gate)")

## 11. Best Practice Comparison: Hardcoded vs Parameterized

Compare hardcoded monolithic design with parameterized modular design. This demonstrates the quality difference between prototype and production code.

In [ ]:
print("Best Practice Comparison")
print("Hardcoded/Monolithic vs Parameterized/Modular\n")

# BAD PRACTICE: Hardcoded monolithic circuit
q0, q1, q2 = cirq.LineQubit.range(3)

hardcoded = cirq.Circuit(
    cirq.H(q0),
    cirq.ry(0.5)(q0),      # Hardcoded angle
    cirq.CNOT(q0, q2),      # Might violate connectivity
    cirq.rz(1.2)(q1),      # Hardcoded angle
    cirq.CNOT(q1, q2),
    cirq.ry(0.8)(q2)       # Hardcoded angle
)

print("BAD: Hardcoded monolithic circuit")
print(hardcoded)
print(f"Is parameterized: {cirq.is_parameterized(hardcoded)}")
print(f"Can optimize: No")
print(f"Connectivity aware: No")

# GOOD PRACTICE: Parameterized modular circuit
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')

qubits = cirq.LineQubit.range(3)

parameterized = cirq.Circuit()

# State preparation layer
parameterized.append(cirq.H.on_each(*qubits))

# Parameterized rotation layer
parameterized.append([
    cirq.ry(alpha)(qubits[0]),
    cirq.ry(beta)(qubits[1]),
    cirq.ry(gamma)(qubits[2])
])

# Entangling layer (nearest-neighbor only)
parameterized.append([
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.CNOT(qubits[1], qubits[2])
])

print("\nGOOD: Parameterized modular circuit")
print(parameterized)
print(f"Is parameterized: {cirq.is_parameterized(parameterized)}")
print(f"Can optimize: Yes ({len(cirq.parameter_names(parameterized))} parameters)")
print(f"Connectivity aware: Yes (nearest-neighbor only)")

print("\nKey differences:")
print("  Hardcoded → Parameterized")
print("  Monolithic → Modular layers")
print("  Connectivity-agnostic → Hardware-aware")
print("  Fixed → Optimizable")

## 12. Production Circuit Checklist

Before deploying circuits to hardware, verify these engineering requirements.

In [ ]:
print("Production Circuit Engineering Checklist")
print("Verify these requirements before hardware deployment\n")

# Example production-ready circuit
qubits = cirq.LineQubit.range(3)
theta = sympy.Symbol('theta')
phi = sympy.Symbol('phi')

production_circuit = cirq.Circuit(
    cirq.H.on_each(*qubits),
    cirq.ry(theta)(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.rz(phi)(qubits[1]),
    cirq.CNOT(qubits[1], qubits[2])
)

print("Example circuit:")
print(production_circuit)

# Checklist validation
connectivity = {0: [1], 1: [0, 2], 2: [1]}

def validate_connectivity(circuit, connectivity):
    for moment in circuit:
        for op in moment:
            if len(op.qubits) == 2:
                q0_idx = op.qubits[0].x
                q1_idx = op.qubits[1].x
                if q1_idx not in connectivity.get(q0_idx, []):
                    return False
    return True

print("\nProduction Readiness Checklist:")
print(f"  ✓ Uses symbolic parameters: {cirq.is_parameterized(production_circuit)}")
print(f"  ✓ Respects connectivity: {validate_connectivity(production_circuit, connectivity)}")

gate_times = {'single': 25e-9, 'two': 50e-9}
exec_time = sum(gate_times['single'] if len(op.qubits) == 1 else gate_times['two'] 
                for moment in production_circuit for op in moment)
t2 = 30e-6
print(f"  ✓ Execution time < T2/10: {exec_time < t2/10}")

depth = len(production_circuit)
print(f"  ✓ Minimized depth: {depth} moments")

print(f"  ✓ Modular design: Uses clear layer structure")
print(f"  ✓ Well-documented: Parameters have clear meanings")

print("\nThis circuit is ready for hardware deployment!")

## Exercises

1. **Connectivity Validation**: Create a circuit that violates linear connectivity (0-1-2-3) and verify it fails validation. Then fix it by inserting SWAP gates.

2. **Parameter Sweep**: Create a parameterized single-qubit rotation circuit. Sweep the parameter from 0 to 2π and plot the measurement probability of |1⟩.

3. **QAOA Design**: Build a 4-qubit QAOA circuit with p=3 layers. How many parameters does it have? What's the circuit depth?

4. **Gate Optimization**: Create a circuit with 5 random single-qubit gates on the same qubit. Use merging to reduce the gate count. What's the reduction ratio?

5. **Decoherence Analysis**: Given T1=100μs and T2=70μs, what's the maximum circuit depth you can afford if single-qubit gates take 20ns and two-qubit gates take 40ns?

6. **Modular Refactoring**: Take a hardcoded 3-qubit circuit and refactor it into: (1) parameterized, (2) modular layers, (3) hardware-aware. Compare depth before and after optimization.

## Key Takeaways

- **Hardware connectivity constraints are real** - validate circuits against device topology or face compilation failures
- **Circuit depth must respect T1/T2 times** - rule of thumb: execution_time < T2/10 for reliable results
- **Native gate sets matter** - understand your hardware to avoid costly decompositions (SWAP = 3 CNOTs!)
- **Always use symbolic parameters (sympy.Symbol)** - never hardcode values in production circuits
- **Modular design scales better** - build circuits from reusable building blocks, not monolithic code
- **Optimize circuit depth with EARLIEST strategy** - parallelize independent gates to reduce depth
- **Gate cancellation and merging are free wins** - use Cirq's optimization utilities automatically
- **Production circuits require engineering discipline** - checklist validation before hardware deployment
- **Best practice: Parameterized + Modular + Hardware-aware + Optimized** - this separates prototype from production code

---

**Next:** [Section 3.3 - Advanced Circuit Optimization](part3_section_3_3_advanced_optimization.ipynb) - Deep dive into compilation, transpilation, and hardware-specific optimization strategies